In [ ]:
from nltk.corpus import wordnet as wn
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import keras
import keras.utils
from keras import utils as np_utils
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [2]:
df=pd.read_csv('labels.csv')
events= pd.read_csv('eventsummary.csv')

In [3]:
df

,event_id,Words,POS,Index,Tag,Labels,Tag2
0,0,A,DT,0,O,0.0,O
1,0,shooting,NN,2,O,0.0,O
2,0,takes,VBZ,11,O,0.0,O
3,0,place,NN,17,O,0.0,O
4,0,at,IN,23,O,0.0,O
...,...,...,...,...,...,...,...
51687,1999,of,IN,167,O,0.0,O
51688,1999,fell,VBD,175,O,0.0,O
51689,1999,Brenham,NNP,183,O,0.0,O
51690,1999,west,NN,191,O,0.0,O


# 1. Small checker 

In [4]:
# for i in range(0,25662):
#     if df.loc[i,'Words']=='Northrop':
#         print("\n")
#         print(i)
#         print(df.loc[i,'event_id'])
        
#         print(df.loc[i,'Words'])
#         print(df.loc[i+1,'Words'])
#         print(df.loc[i+2,'Words'])
#         print(df.loc[i+3,'Words'])
    

In [5]:
events=events[0:2000]

In [6]:
corpus=events['event_summary']

In [7]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t,m) for w, p, t,m in zip(s["Words"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist(),
                                                           s["Tag2"].values.tolist())]
        self.grouped = self.data.groupby("event_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

In [8]:
word_tokenizer = Tokenizer()
print(word_tokenizer.fit_on_texts(corpus))
vocab_length = len(word_tokenizer.word_index) + 1
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

print(length_long_sentence)

None
77


In [9]:
X=padded_sentences
max_len=length_long_sentence
# words = set(df["Words"].values.tolist())
# # words.append("ENDPAD")
# n_words = len(words); 
tags = set(df["Tag"].values.tolist())
n_tags = len(tags); 
tags2 = set(df["Tag2"].values.tolist())
n_tags2 = len(tags2); 
sent = getter.get_next()
# word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)} #vocab for tags arguments
tag22idx = {t: i for i, t in enumerate(tags2)} #vocab for tags relation
# from keras.preprocessing.sequence import pad_sequences
# X = [[word2idx[w[0]] for w in s] for s in sentences]
# X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
y = [to_categorical(i, num_classes=n_tags) for i in y]
#.....................output 2 label Relations..........................
y1 = [[tag22idx[w[3]] for w in s] for s in sentences]
y1 = pad_sequences(maxlen=max_len, sequences=y1, padding="post", value=tag22idx["O"])
y1 = [to_categorical(i, num_classes=n_tags2) for i in y1]
# print(y)

In [10]:
# X[0]

In [11]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('C:/Users/gulja/Python 3.7.3/Scripts/thesis/2020/Word Embeddings/glove.6B.100d.txt', encoding="utf8")


In [12]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [13]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [14]:
embedding_matrix 

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.1529    , -0.24279   ,  0.89837003, ..., -0.59100002,
         1.00390005,  0.20664001],
       [ 0.085703  , -0.22201   ,  0.16569   , ..., -0.074273  ,
         0.75808001, -0.34243   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.50278002,  0.64444   , -0.044803  , ..., -0.11322   ,
         0.63252997, -0.82796001],
       [ 0.28566   , -0.12547   ,  0.061527  , ...,  0.10364   ,
         0.034506  ,  0.14213   ]])

In [15]:
len(X)

2000

In [16]:
y=y[0:2000]
y1=y1[0:2000]

In [17]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [18]:
X_tr, X_te, y_tr, y_te,y1_tr, y1_te = train_test_split(X,y, y1, test_size=0.1)

In [ ]:
input = Input(shape=(max_len,))
model = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="sigmoid"))(model)  # softmax output layer
out1 = TimeDistributed(Dense(n_tags2, activation="sigmoid"))(model) 
model = Model(input, [out,out1])
model.compile(optimizer="adam",lr=0.01, loss="binary_crossentropy", metrics=["accuracy",f1_m,precision_m, recall_m])

In [20]:
from keras.utils.vis_utils import plot_model

In [21]:
# import os
# os.environ["PATH"] += os.pathsep + 'C:/Users/gulja/Python 3.7.3/Lib/site-packages/graphviz/'

In [23]:
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
# What I did are followed:
model.summary()
import keras
import pydot_ng as pydot
pydot.find_graphviz()
plot_model(model, to_file='model_plot.png', show_shapes=False, show_layer_names=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 77)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 77, 100)      985400      input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 77, 100)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 77, 200)      160800      dropout_1[0][0]                  
__________________________________________________________________________________________________
time_distr

In [ ]:
history = model.fit(X_tr, [np.array(y_tr),np.array(y1_tr)], batch_size=32, epochs=50, validation_split=0.2, verbose=1)


In [ ]:
hist = pd.DataFrame(history.history)
# print(hist)

In [ ]:
hist.loc[49]

In [ ]:
# hist

In [ ]:
from matplotlib import pyplot
pyplot.subplot(211)
pyplot.title('F1-Score')
pyplot.plot(history.history['time_distributed_2_f1_m'], label='train')
pyplot.plot(history.history['val_time_distributed_2_f1_m'], label='test')
pyplot.legend()

In [ ]:
X_te

In [ ]:
i = 112
p = model.predict(np.array([X_te[i]]))
p = np.argmax(p, axis=-1)
print('.....................................................................................')
print(p)
# print("{:15} ({:5}): {}".format("Word", "True", "Pred"))
# for w, pred in zip(X_te[i], p[0][0]):
#     print("{:15}: {}".format(embedding_matrix[w], list(tags)[pred]))
# print('.....................................................................................')    
# print(p[1][0])
# for w, pred1 in zip(X_te[i], p[1][0]):
#     print("{:15}: {}".format(list(words)[w], list(tags2)[pred1]))

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


In [ ]:
test_pred = model.predict(X_te, verbose=1)


In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [ ]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))


In [ ]:
print(classification_report(test_labels, pred_labels))
